In [2]:
import numpy as np
from helpers.icq_methods import *

In [3]:
# Our current version using Numpy
vectorX = [-1, -0.5]
vectorW = [-1, -0.5]

sigmaQ = get_sigmaQ(2)
sigmaE = get_sigmaE(vectorX, vectorW)

U_operator = get_U_operator(sigmaQ, sigmaE)

N = len(vectorX)

p_cog = get_p([[1/np.sqrt(2)],[1/np.sqrt(2)]])
p_env = get_p([[1/np.sqrt(N)] for i in range(N)])

quantum_operation = np.array(U_operator * (np.kron(p_cog, p_env)) * U_operator.getH())
p_cog_new = np.trace(quantum_operation.reshape([2,N,2,N]), axis1=1, axis2=3)
p_cog_new_00_2 = p_cog_new[0,0]
np.real(np.round(p_cog_new_00_2, 6))

0.755919

In [4]:
import sympy
from sympy import Matrix, Symbol, exp, simplify, lambdify
from sympy.physics.quantum.trace import Tr
from sympy.physics.quantum.dagger import Dagger
from sympy.physics.quantum import TensorProduct
from sympy.physics.quantum.density import Density

In [5]:
# Our current version using Sympy
vector_i = [Symbol('a'), Symbol('b')]
vector_w = [Symbol('A'), Symbol('B')]

N = len(vector_i)

sigma_q_sympy = Matrix([
    [1, (1 - 1j)],
    [(1j + 1), -1]
])

sigma_e_sympy = Matrix(np.zeros((N, N)))
for index_i in range(N):
        sigma_e_sympy[index_i,index_i] = vector_i[index_i] * vector_w[index_i]
sigma_e_sympy = Matrix(sigma_e_sympy)

u_operator_sympy = Matrix(1j*TensorProduct(sigma_q_sympy, sigma_e_sympy)).exp()

p_cog = Matrix(np.ones((2, 2)) * (1/2))

# Two times 1/sqrt(N) because p_env = |p_env><p_env|, which will apply the amplitude twice
p_env = np.ones((N, N))
p_env = 1/np.sqrt(N) * 1/np.sqrt(N) * p_env
p_env = Matrix(p_env)

quantum_operation_sympy = u_operator_sympy * TensorProduct(p_cog, p_env) * Dagger(u_operator_sympy)

# We need to do a partial trace using p_env (Eq. 20)

# Using numpy, we can do a similar approach to what we have rn:
# First turn our sympy matrix into a numpy matrix by adding its values
numpy_matrix_fun = lambdify([vector_i[0], vector_i[1], vector_w[0], vector_w[1]], quantum_operation_sympy)
numpy_matrix = np.matrix(numpy_matrix_fun(-1, -0.5, -1, -0.5))
numpy_matrix = np.array(numpy_matrix).astype(np.complex_)

# and calculate using the np.trace
p_cog_new = np.trace(numpy_matrix.reshape([2,N,2,N]), axis1=1, axis2=3)
p_cog_new_00_2 = p_cog_new[0,0]

# ofc we need to get only the real part
print(np.round(np.real(p_cog_new_00_2), 6))

# As for doing it via Sympy, it's not so simple.
# As Eq.20 suggests, we need to do the partial trace according to rho_env,
# so I'm trying to create a new Density that combines both our system quantum state 
# (quantum_operation_sympy) and rho_env (p_env), and then getting the Trace on the second part
density = Density([(quantum_operation_sympy, 1), (p_env, 1)])
Tr(density, [1, 1])

0.755919


Tr(((((0.788675134594813*(-0.366025403784439 - 0.366025403784439*I)*exp(1.73205080756888*I*conjugate(A)*conjugate(a)) + 0.211324865405187*(1.36602540378444 + 1.36602540378444*I)*exp(-1.73205080756888*I*conjugate(A)*conjugate(a)))*(0.25*(0.288675134594813 + 0.288675134594813*I)*(1.36602540378444 - 1.36602540378444*I)*exp(1.73205080756888*I*A*a) + 0.0528312163512968*(1.36602540378444 - 1.36602540378444*I)*exp(1.73205080756888*I*A*a) + 0.25*(-0.366025403784439 + 0.366025403784439*I)*(-0.288675134594813 - 0.288675134594813*I)*exp(-1.73205080756888*I*A*a) + 0.197168783648703*(-0.366025403784439 + 0.366025403784439*I)*exp(-1.73205080756888*I*A*a)) + ((-0.366025403784439 - 0.366025403784439*I)*(-0.288675134594813 + 0.288675134594813*I)*exp(1.73205080756888*I*conjugate(A)*conjugate(a)) + (0.288675134594813 - 0.288675134594813*I)*(1.36602540378444 + 1.36602540378444*I)*exp(-1.73205080756888*I*conjugate(A)*conjugate(a)))*(0.25*(0.288675134594813 + 0.288675134594813*I)*(1.36602540378444 - 1.36602